In [1]:
import numpy as np 
import pandas as pd
from __future__ import division

In [2]:
#Load csv
game_data = pd.read_csv('nba_2017_2018_games.csv')

In [3]:
#List of Eastern Conference teams
east_teams = ['Toronto Raptors',
'Boston Celtics',
'Philadelphia 76ers',
'Cleveland Cavaliers',
'Indiana Pacers',
'Miami Heat',
'Milwaukee Bucks',
'Washington Wizards',
'Detroit Pistons',
'Charlotte Hornets',
'New York Knicks',
'Brooklyn Nets',
'Chicago Bulls',
'Orlando Magic',
'Atlanta Hawks']

atlantic = ['Toronto Raptors',
'Boston Celtics',
'Philadelphia 76ers',
'New York Knicks',
'Brooklyn Nets']

central = ['Cleveland Cavaliers',
'Indiana Pacers',
'Milwaukee Bucks',
'Detroit Pistons',
'Chicago Bulls']

pacific = ['Golden State Warriors',
'Los Angeles Clippers',
'Los Angeles Lakers',
'Sacramento Kings',
'Phoenix Suns']

northwest= ['Denver Nuggets',
'Oklahoma City Thunder',
'Portland Trail Blazers',
'Utah Jazz',
'Minnesota Timberwolves']

In [4]:
#Get winners/losers of each game and assign points for OT losses
game_data['Winner'] = np.where(game_data['PTS'] > game_data['PTS.1'], game_data['Visitor/Neutral'], game_data['Home/Neutral'])
game_data['Loser'] = np.where(game_data['PTS'] < game_data['PTS.1'], game_data['Visitor/Neutral'], game_data['Home/Neutral'])
game_data['Winner_pts'] = 2
game_data['Loser_pts'] = np.where(game_data['OT'].isnull(), 0, 1)

In [5]:
#Build DataFrames for winner and loser points
winner_totals = pd.DataFrame(game_data.groupby('Winner')['Winner_pts'].sum().reset_index().rename(columns={'Winner':'Team'}))
loser_totals = pd.DataFrame(game_data.groupby('Loser')['Loser_pts'].sum().reset_index().rename(columns={'Loser':'Team'}))

In [6]:
#Make new DataFrame from winner and loser tables
point_totals = winner_totals.merge(loser_totals)
point_totals['Total_points'] = point_totals['Winner_pts'] + point_totals['Loser_pts']
point_totals['Point_fraction'] = point_totals['Total_points']/164
point_totals['Conference'] = np.where(point_totals['Team'].isin(east_teams), 'East', 'West')
point_totals['Division'] = np.where((point_totals['Conference'] == 'East') & (point_totals['Team'].isin(atlantic)), 'Atlantic', np.where(point_totals['Team'].isin(central), 'Central', 'Southeast'))
point_totals['Division'] = np.where((point_totals['Conference'] == 'West') & (point_totals['Team'].isin(pacific)), 'Pacific', np.where(point_totals['Team'].isin(northwest), 'Northwest', 'Southwest'))

In [7]:
#Add column for division
point_totals['Division'] = np.where(point_totals['Conference'] == 'East', np.where(point_totals['Team'].isin(atlantic), 'Atlantic', np.where(point_totals['Team'].isin(central), 'Central', 'Southeast')), None)
point_totals['Division'] = np.where(point_totals['Conference'] == 'West', np.where(point_totals['Team'].isin(pacific), 'Pacific', np.where(point_totals['Team'].isin(northwest), 'Northwest', 'Southwest')), point_totals['Division'])

In [8]:
#entire league standings
point_totals = point_totals.sort_values(by=['Total_points'], ascending=False)
point_totals.index = np.arange(1, len(point_totals) + 1)
point_totals.style.apply(lambda x: ['background: lightgreen' if x.name in range(1,17) else '' for i in x], axis=1)

,Team,Winner_pts,Loser_pts,Total_points,Point_fraction,Conference,Division
1,Houston Rockets,130,0,130,0.792683,West,Southwest
2,Toronto Raptors,118,2,120,0.731707,East,Atlantic
3,Golden State Warriors,116,0,116,0.707317,West,Pacific
4,Boston Celtics,110,2,112,0.682927,East,Atlantic
5,Philadelphia 76ers,104,1,105,0.640244,East,Atlantic
6,Cleveland Cavaliers,100,0,100,0.609756,East,Central
7,Portland Trail Blazers,98,1,99,0.603659,West,Northwest
8,New Orleans Pelicans,96,2,98,0.597561,West,Southwest
9,Utah Jazz,96,1,97,0.591463,West,Northwest
10,Oklahoma City Thunder,96,1,97,0.591463,West,Northwest


In [9]:
#Build Eastern and Western conference DataFrames 
conference_mask = point_totals['Conference'].isin(['East'])
east_totals = point_totals[conference_mask]
west_totals = point_totals[~conference_mask]

In [10]:
west_totals = west_totals.sort_values(by=['Division', 'Total_points'], ascending=False)
west_totals.style.apply(lambda x: ['background: lightgreen' if x.name in [1, 8, 3, 18, 7, 9] else ('background: gold' if x.name in [10, 11] else '') for i in x], axis=1)

,Team,Winner_pts,Loser_pts,Total_points,Point_fraction,Conference,Division
1,Houston Rockets,130,0,130,0.792683,West,Southwest
8,New Orleans Pelicans,96,2,98,0.597561,West,Southwest
14,San Antonio Spurs,94,1,95,0.579268,West,Southwest
26,Dallas Mavericks,48,5,53,0.323171,West,Southwest
29,Memphis Grizzlies,44,1,45,0.27439,West,Southwest
3,Golden State Warriors,116,0,116,0.707317,West,Pacific
18,Los Angeles Clippers,84,1,85,0.518293,West,Pacific
20,Los Angeles Lakers,70,5,75,0.457317,West,Pacific
25,Sacramento Kings,54,0,54,0.329268,West,Pacific
30,Phoenix Suns,42,1,43,0.262195,West,Pacific


In [11]:
east_totals = east_totals.sort_values(by=['Division', 'Total_points'], ascending=False)
east_totals.style.apply(lambda x: ['background: lightgreen' if x.name in [15, 17, 6, 12, 2, 4] else ('background: gold' if x.name in [5, 16] else '') for i in x], axis=1)

,Team,Winner_pts,Loser_pts,Total_points,Point_fraction,Conference,Division
15,Miami Heat,88,6,94,0.573171,East,Southeast
17,Washington Wizards,86,3,89,0.542683,East,Southeast
21,Charlotte Hornets,72,2,74,0.45122,East,Southeast
27,Orlando Magic,50,0,50,0.304878,East,Southeast
28,Atlanta Hawks,48,1,49,0.29878,East,Southeast
6,Cleveland Cavaliers,100,0,100,0.609756,East,Central
12,Indiana Pacers,96,0,96,0.585366,East,Central
16,Milwaukee Bucks,88,3,91,0.554878,East,Central
19,Detroit Pistons,78,4,82,0.5,East,Central
24,Chicago Bulls,54,3,57,0.347561,East,Central


In [12]:
#Output en Easter conference final standings
east_totals = east_totals.loc[[15, 17, 6, 12, 2, 4, 5, 16]].sort_values(by=['Total_points'], ascending=False)
east_totals = east_totals.sort_values(by=['Total_points'], ascending=False)
east_totals.index = np.arange(1, len(east_totals) + 1)
east_totals

,Team,Winner_pts,Loser_pts,Total_points,Point_fraction,Conference,Division
1,Toronto Raptors,118,2,120,0.731707,East,Atlantic
2,Boston Celtics,110,2,112,0.682927,East,Atlantic
3,Philadelphia 76ers,104,1,105,0.640244,East,Atlantic
4,Cleveland Cavaliers,100,0,100,0.609756,East,Central
5,Indiana Pacers,96,0,96,0.585366,East,Central
6,Miami Heat,88,6,94,0.573171,East,Southeast
7,Milwaukee Bucks,88,3,91,0.554878,East,Central
8,Washington Wizards,86,3,89,0.542683,East,Southeast


In [13]:
#Output Western conference final standings
west_totals = west_totals.loc[[1, 8, 3, 18, 7, 9, 10, 11]].sort_values(by=['Total_points'], ascending=False)
west_totals = west_totals.sort_values(by=['Total_points'], ascending=False)
west_totals.index = np.arange(1, len(west_totals) + 1)
west_totals

,Team,Winner_pts,Loser_pts,Total_points,Point_fraction,Conference,Division
1,Houston Rockets,130,0,130,0.792683,West,Southwest
2,Golden State Warriors,116,0,116,0.707317,West,Pacific
3,Portland Trail Blazers,98,1,99,0.603659,West,Northwest
4,New Orleans Pelicans,96,2,98,0.597561,West,Southwest
5,Utah Jazz,96,1,97,0.591463,West,Northwest
6,Oklahoma City Thunder,96,1,97,0.591463,West,Northwest
7,Denver Nuggets,92,4,96,0.585366,West,Northwest
8,Los Angeles Clippers,84,1,85,0.518293,West,Pacific
